# Testing FDM run_models

## Setup

Let's start by setting up the simulation environment.

In [1]:
import os
import jsbsim

import sys
sys.path.insert(0, './utils/')
from JSBSim_utils import CheckXMLFile

# Relative path to the directory where the flight model is stored
# Note - Aircraft directory needs to be writeable in order to modify the cg
PATH_TO_JSBSIM_FILES="../.."

print(f'=================================================================')
print(f'Current working directory: {os.getcwd()}')

# Global variables that must be modified to match your particular need
# The aircraft name
# Note - It should match the exact spelling of the model file
AIRCRAFT_NAME="global5000"

# The path to the input file c172c.xml
fdm_input_file_path = os.path.join(PATH_TO_JSBSIM_FILES, "aircraft", AIRCRAFT_NAME, f"{AIRCRAFT_NAME}.xml")

print(f'Input file path: {fdm_input_file_path}')

chk_fdm_file = CheckXMLFile(fdm_input_file_path, header='fdm_config')

# printe the check result
print(f'FDM input file check success: {chk_fdm_file}') 

# Console with log messages
jsbsim.FGJSBBase().debug_lvl = 0

# Instantiate the FDMExec object and load the aircraft
fdm = jsbsim.FGFDMExec(PATH_TO_JSBSIM_FILES)

load_status = fdm.load_model(AIRCRAFT_NAME)

# Set the output path to the current directory
fdm.set_output_path('examples/python/')

print(f'\nAircraft FDM: {AIRCRAFT_NAME}\nLoaded: {load_status}')
print(fdm)
print(f'=================================================================')

Current working directory: f:\agodemar\jsbsim\examples\python
Input file path: ../..\aircraft\global5000\global5000.xml
FDM input file check success: True

Aircraft FDM: global5000
Loaded: True
FGFDMExec 
root dir	:	../..
aircraft path	:	../../aircraft
engine path	:	../../engine
systems path	:	../../systems
output path	:	../../examples/python



## Explore the FDM catalog of properties

In [2]:
# Get the aircraft catalog of properties
# Part of them is defined internally by JSBSim, and part of them is defined in the FDM input file
catalog = fdm.get_property_catalog()
# catalog is a list of strings, print the entries
for entry in catalog:
    print(entry)

inertial/sea-level-radius_ft (R)
simulation/gravity-model (RW)
simulation/integrator/rate/rotational (RW)
simulation/integrator/rate/translational (RW)
simulation/integrator/position/rotational (RW)
simulation/integrator/position/translational (RW)
simulation/write-state-file (W)
simulation/channel-dt (R)
simulation/gravitational-torque (RW)
simulation/force-output (W)
simulation/do_simple_trim (W)
simulation/do_linearization (W)
simulation/reset (W)
simulation/disperse (R)
simulation/randomseed (RW)
simulation/terminate (RW)
simulation/pause (RW)
simulation/sim-time-sec (R)
simulation/dt (R)
simulation/jsbsim-debug (RW)
simulation/frame (RW)
simulation/trim-completed (RW)
simulation/output/log_rate_hz (RW)
simulation/output/enabled (RW)
velocities/h-dot-fps (R)
velocities/v-north-fps (R)
velocities/v-east-fps (R)
velocities/v-down-fps (R)
velocities/u-fps (R)
velocities/v-fps (R)
velocities/w-fps (R)
velocities/p-rad_sec (R)
velocities/q-rad_sec (R)
velocities/r-rad_sec (R)
velocities

## Simulation setup

In [3]:
import xml.etree.ElementTree as ET

# Fuel max for Global5000, lbm
fuelmax = 8097.63

# Instantiate the FDMExec object and load the aircraft
fdm = jsbsim.FGFDMExec(PATH_TO_JSBSIM_FILES)
fdm.load_model(AIRCRAFT_NAME)

ac_xml_tree = ET.parse(os.path.join(fdm.get_root_dir(), f'aircraft/{AIRCRAFT_NAME}/{AIRCRAFT_NAME}.xml'))
ac_xml_root = ac_xml_tree.getroot()

# Get the empty weight from aircraft xml [assume lbs]
for x in ac_xml_root.findall('mass_balance'):
        w = x.find('emptywt').text

empty_weight = float(w)

# Get the original CG from aircraft xml, assum inches from Construction axes origin
for loc in ac_xml_root.findall('mass_balance/location'):
        x_cg_ = loc.find('x').text

x_cg_0 = float(x_cg_)

# Assume a payload, midweight, lb
payload_0 = 15172/2

# Assume the mass of fuel, half tanks, lb
fuel_per_tank = fuelmax/2

# Assume a flight altitude
h_ft_0 = 15000

weight_0 = empty_weight + payload_0 + fuel_per_tank*3

# Assume a zero flight path angle, gamma_0
gamma_0 = 0

print("-----------------------------------------")
print("Altitude {} ft, Weight {} lb, CoG-x {} in".format(h_ft_0, weight_0, x_cg_0))
print("-----------------------------------------")

# Set engines running
fdm['propulsion/set-running'] = -1

# Set required initial CAS, kts
speed_0 = 250

# --- unknown trim variables, set to None

# 'forces/fbx-total-lbs'
fxb_total_lbs_0 = None
# 'forces/fyb-total-lbs'
fyb_total_lbs_0 = None
# 'forces/fzb-total-lbs'
fzb_total_lbs_0 = None

# 'fcs/elevator-pos-rad'
delta_e_0 = None
# 'fcs/elevator-pos-norm'
delta_e_cmd_0 = None
# 'fcs/aileron-pos-rad'
delta_a_0 = None
# 'fcs/aileron-pos-norm'
delta_a_cmd_0 = None
# 'fcs/rudder-pos-rad'
delta_r_0 = None
# 'fcs/rudder-pos-norm'
delta_r_cmd_0 = None
# 'fcs/throttle-cmd-norm[0]'
throttle_cmd0_0 = None
# 'fcs/throttle-cmd-norm[1]'
throttle_cmd1_0 = None

# 'aero/alpha-rad'
alpha_0 = None
# 'aero/beta-rad'
beta_0 = None

fdm['ic/h-sl-ft']                      = h_ft_0
fdm['ic/vc-kts']                       = speed_0
fdm['ic/gamma-deg']                    = gamma_0
fdm['ic/beta-deg']                     = 0
fdm['propulsion/tank[0]/contents-lbs'] = fuel_per_tank
fdm['propulsion/tank[1]/contents-lbs'] = fuel_per_tank
fdm['propulsion/tank[2]/contents-lbs'] = fuel_per_tank
fdm['inertia/pointmass-weight-lbs[0]'] = payload_0

# Initialize the aircraft with initial conditions
fdm.run_ic()

pm = fdm.get_property_manager()

sim_time_node = pm.get_node("simulation/sim-time-sec")
sim_time = sim_time_node.get_double_value()
print(f"Initial sim time: {sim_time} sec")

# Trim
try:
    fdm['simulation/do_simple_trim'] = 1

    alpha_0 = pm.get_node('aero/alpha-rad').get_double_value()
    beta_0 = pm.get_node('aero/beta-rad').get_double_value()
    delta_e_0 = pm.get_node('fcs/elevator-pos-rad').get_double_value()
    delta_e_cmd_0 = pm.get_node('fcs/elevator-pos-norm').get_double_value()
    throttle_cmd0_0 = pm.get_node('fcs/throttle-cmd-norm[0]').get_double_value()
    throttle_cmd1_0 = pm.get_node('fcs/throttle-cmd-norm[1]').get_double_value()

    # --- Now that trim is done, we can get the forces and moments, which should be close to zero

    do_run_all_models = False
    if do_run_all_models:
        # Run all models
        print("\nRunning all models...")
        fdm_status = fdm.run()
    else:
        # Run only selected models
        print("\nRunning only selected models...")
        # Testing fdm.run_models to run only specific models. This is useful to speed up the simulation when only a subset of the models is needed for the analysis.
        """ Execute a simulation step running only the specified models.
        Instead of running all models, only the models whose indices are
        provided in *selected_models* are executed.  Use the ``eModels``
        enum values to identify each model:
         0: ePropagate
         1: eInput
         2: eInertial
         3: eAtmosphere
         4: eWinds
         5: eSystems
         6: eMassBalance
         7: eAuxiliary
         8: ePropulsion
         9: eAerodynamics
        10: eGroundReactions
        11: eExternalReactions
        12: eBuoyantForces
        13: eAircraft
        14: eAccelerations
        15: eOutput
          :param selected_models: A list of model indices to run.
          :return: True if successful, False if the sim should end.
        """
        fdm_status = fdm.run_models([
                #0,  # Propagate
                #1,  # Inputs
                #3,  # Atmosphere
                #8,  # Propulsion
                #9,  # Aerodynamics
                #13, # Aircraft
                14   # Accelerations
                ])

    print(f"Simulation step status: {fdm_status}")

    sim_time = sim_time_node.get_double_value()
    print(f"Initial sim time: {sim_time} sec")

    udot_node = pm.get_node("accelerations/udot-ft_sec2")
    vdot_node = pm.get_node("accelerations/vdot-ft_sec2")
    wdot_node = pm.get_node("accelerations/wdot-ft_sec2")
    pdot_node = pm.get_node("accelerations/pdot-rad_sec2")
    qdot_node = pm.get_node("accelerations/qdot-rad_sec2")
    rdot_node = pm.get_node("accelerations/rdot-rad_sec2")

    thrust_lbs_0_node = pm.get_node('propulsion/engine/thrust-lbs')

    print("\nTrimmed flight condition:")
    print("Altitude {} ft, Weight {} lb, CoG-x {} in".format(h_ft_0, weight_0, x_cg_0))
    print("Trimmed alpha {} deg, beta {} deg".format(alpha_0*180/3.14159, beta_0*180/3.14159))
    print("Trimmed elevator pos {} deg, elevator cmd {} norm".format(delta_e_0*180/3.14159, delta_e_cmd_0))
    print("Trimmed throttle cmd 0: {} norm".format(throttle_cmd0_0))
    print("Trimmed throttle cmd 1: {} norm".format(throttle_cmd1_0))
    
    print("Trimmed thrust: {} lbs".format(thrust_lbs_0_node))

    print("Trimmed udot: {} ft/s^2".format(udot_node.get_double_value()))
    print("Trimmed vdot: {} ft/s^2".format(vdot_node.get_double_value()))
    print("Trimmed wdot: {} ft/s^2".format(wdot_node.get_double_value()))

    print("Trimmed pdot: {} rad/s^2".format(pdot_node.get_double_value()))
    print("Trimmed qdot: {} rad/s^2".format(qdot_node.get_double_value()))
    print("Trimmed rdot: {} rad/s^2".format(rdot_node.get_double_value()))

    print("-----------------------------------------")

except jsbsim.TrimFailureError:
    print("Trim failure, skipping this point")
    pass  # Ignore trim failure exceptions

-----------------------------------------
Altitude 15000 ft, Weight 67967.445 lb, CoG-x 790.82 in
-----------------------------------------
Initial sim time: 0.0 sec

Running only selected models...
Simulation step status: True
Initial sim time: 0.008333333333333333 sec

Trimmed flight condition:
Altitude 15000 ft, Weight 67967.445 lb, CoG-x 790.82 in
Trimmed alpha 4.317587370576376 deg, beta -8.523964522074272e-15 deg
Trimmed elevator pos -2.963416710073272 deg, elevator cmd -0.14777524289204905 norm
Trimmed throttle cmd 0: 0.7726100117418303 norm
Trimmed throttle cmd 1: 0.7726100117418303 norm
Trimmed thrust: Property '/fdm/jsbsim/propulsion/engine/thrust-lbs' (value: 5387.515518261779) lbs
Trimmed udot: -0.0001934079236165509 ft/s^2
Trimmed vdot: 3.550866362780659e-14 ft/s^2
Trimmed wdot: 0.00017606913795020773 ft/s^2
Trimmed pdot: -3.4846824985750854e-16 rad/s^2
Trimmed qdot: -2.4192836093322397e-10 rad/s^2
Trimmed rdot: 5.956952464448566e-18 rad/s^2
-------------------------------

In [5]:
# Change elevator deflection and see how the accelerations change
delta_e_cmd = fdm['fcs/elevator-cmd-norm']

print("\nChanging elevator command by +0.3 norm...")
delta_e_cmd = delta_e_cmd_0 + 0.3
fdm['fcs/elevator-cmd-norm'] = delta_e_cmd
print("New elevator cmd {} norm".format(delta_e_cmd))
print("Current elevator cmd {} norm".format(pm.get_node('fcs/elevator-cmd-norm').get_double_value()))

delta_e = pm.get_node('fcs/elevator-pos-rad').get_double_value()
#delta_e_cmd = pm.get_node('fcs/elevator-pos-norm').get_double_value()

print("Current elevator pos {} deg".format(pm.get_node('fcs/elevator-pos-deg').get_double_value()))


# Update the simulation to see how the accelerations change
do_run_all_models = True
if do_run_all_models:
    # Run all models
    print("\nRunning all models...")
    fdm_status = fdm.run()
else:
    # Run only selected models
    print("\nRunning only selected models...")

    fdm_status = fdm.run_models([
        #0, # Propagate
         1, # Inputs
        #3, # Atmosphere
        #8, # Propulsion
         9, # Aerodynamics
        13, # Aircraft
        14  # Accelerations
    ])

print(f"Simulation step status: {fdm_status}")

sim_time = sim_time_node.get_double_value()
print(f"sim time: {sim_time} sec")

# Get the updated accelerations
udot_node = pm.get_node("accelerations/udot-ft_sec2")
vdot_node = pm.get_node("accelerations/vdot-ft_sec2")
wdot_node = pm.get_node("accelerations/wdot-ft_sec2")
pdot_node = pm.get_node("accelerations/pdot-rad_sec2")
qdot_node = pm.get_node("accelerations/qdot-rad_sec2")
rdot_node = pm.get_node("accelerations/rdot-rad_sec2")
print("Trimmed udot: {} ft/s^2".format(udot_node.get_double_value()))
print("Trimmed vdot: {} ft/s^2".format(vdot_node.get_double_value()))
print("Trimmed wdot: {} ft/s^2".format(wdot_node.get_double_value()))
print("Trimmed pdot: {} rad/s^2".format(pdot_node.get_double_value()))
print("Trimmed qdot: {} rad/s^2".format(qdot_node.get_double_value()))
print("Trimmed rdot: {} rad/s^2".format(rdot_node.get_double_value()))



Changing elevator command by +0.3 norm...
New elevator cmd 0.15222475710795094 norm
Current elevator cmd 0.15222475710795094 norm
Current elevator pos 0.08922843490912412 deg

Running all models...
Simulation step status: True
sim time: 0.025 sec
Trimmed udot: 0.6883591895367411 ft/s^2
Trimmed vdot: -1.936146091280653e-06 ft/s^2
Trimmed wdot: -1.925782170543755 ft/s^2
Trimmed pdot: 4.4715408558505446e-10 rad/s^2
Trimmed qdot: -0.2080685169204454 rad/s^2
Trimmed rdot: 1.8087063724477888e-07 rad/s^2
